# 2.3 Subconsultas Avanzadas\n\n- Objetivo: aplicar subconsultas correlacionadas y EXISTS.\n- Prerrequisitos: joins y agregaciones sólidas.\n- Ejercicios: comparación vs promedio global, EXISTS con condición.\n- Reto: reescritura con CTE y JOIN para legibilidad y rendimiento.\n- Errores comunes: correlaciones profundas, subconsultas innecesarias, falta de índices en filtros.

In [ ]:
-- Subconsulta correlacionada: productos con margen total > promedio global
SELECT p.producto_id, p.nombre
FROM dbo.dim_productos p
WHERE (SELECT SUM(fv.cantidad*(p.precio_unitario-p.costo_unitario))
       FROM dbo.fact_ventas fv
       WHERE fv.producto_id = p.producto_id) > (
          SELECT AVG(sub.margen_prod)
          FROM (SELECT producto_id, SUM(cantidad*(precio_unitario-costo_unitario)) AS margen_prod
                FROM dbo.fact_ventas fv2
                JOIN dbo.dim_productos p2 ON fv2.producto_id = p2.producto_id
                GROUP BY producto_id) sub
      );

In [ ]:
-- EXISTS: productos con al menos una venta con descuento >30%
SELECT p.producto_id, p.nombre
FROM dbo.dim_productos p
WHERE EXISTS (SELECT 1 FROM dbo.fact_ventas fv WHERE fv.producto_id = p.producto_id AND fv.descuento_pct > 30);

🟢 Ejercicio: Productos con ventas en todos los días registrados (sugerencia: comparar count(distinct fecha)).

In [ ]:
SELECT p.producto_id, p.nombre
FROM dbo.dim_productos p
WHERE (SELECT COUNT(DISTINCT fecha) FROM dbo.fact_ventas fv WHERE fv.producto_id = p.producto_id) = (SELECT COUNT(DISTINCT fecha) FROM dbo.fact_ventas);

🟠 Ejercicio: Clientes cuyo total de unidades supera el promedio de su segmento (subconsulta correlacionada con segmento).

In [ ]:
SELECT c.cliente_id, c.nombre, c.segmento
FROM dbo.dim_clientes c
WHERE (SELECT SUM(fv.cantidad) FROM dbo.fact_ventas fv WHERE fv.cliente_id = c.cliente_id) > (
    SELECT AVG(sub.total_unidades)
    FROM (SELECT cliente_id, SUM(cantidad) AS total_unidades
          FROM dbo.fact_ventas fv2
          GROUP BY cliente_id) sub
    JOIN dbo.dim_clientes c2 ON sub.cliente_id = c2.cliente_id
    WHERE c2.segmento = c.segmento
);

🔴 Reto: Reescribir ejercicio anterior con CTE y JOIN evitando subconsulta correlacionada para mejorar legibilidad y potencial rendimiento.

Errores comunes: subconsultas innecesarias vs JOIN; performance degradado por correlaciones profundas; olvidar índices en columnas filtradas.

# Créditos

Este material fue revisado y enriquecido parcialmente mediante asistencia de IA (OpenAI y Claude); la validación y decisiones editoriales finales son humanas.